In [1]:
#from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import random
import tensorflow as tf

from gensim.models import Word2Vec

In [2]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential 
#from tensorflow.keras.models import Sequential, load_model
#from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras import backend

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import LancasterStemmer

nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
data = pd.read_csv("/content/drive/My Drive/NLP/Project/NLPMiniProject_data.csv")
print(data.head())

           ID                                            comment     date  \
0  uid_590555  Well, let's be honest here, they don't actuall...  2015-04   
1  uid_671762  Well, I didn't need evidence to believe in com...  2016-12   
2  uid_519689              Who does an "official promo" in 360p?  2013-11   
3  uid_788362                           Grotto koth was the best  2015-09   
4  uid_299252                                   Neal's back baby  2015-11   

   down                                     parent_comment  score  top  \
0     0  They should shut the fuck up and let the commu...      2    2   
1    -1  You need evidence to kill people? I thought we...      6   -1   
2     0                    2014 BMW S1000R: Official Promo      3    3   
3     0  Not really that memorable lol if you want memo...      2    2   
4     0                      James Neal hit on Zach Parise     -5   -5   

              topic            user  label  
0       starcitizen  Combat_Wombatz      0  
1 

In [11]:
df = data[['ID','comment','parent_comment','down','score','top','topic','label']]

In [12]:
df['comment'][0]

"Well, let's be honest here, they don't actually seem to do much moderating, so they have to spend their time doing *something*."

In [13]:
df['parent_comment'][0]

'They should shut the fuck up and let the community decide what it wants for the subreddit. They are here to **moder** fucking **ate**, not to dictate.'

In [14]:
(df['comment']+df['parent_comment'])[0]

"Well, let's be honest here, they don't actually seem to do much moderating, so they have to spend their time doing *something*.They should shut the fuck up and let the community decide what it wants for the subreddit. They are here to **moder** fucking **ate**, not to dictate."

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    df['comment']+df['parent_comment'],
    df['label'],
    test_size=0.2, 
    random_state=42
)

In [16]:
X_train.shape, X_test.shape

((12000,), (3000,))

# Build the Tokenizer

In [17]:
desired_vocab_size = 10000 #Vocablury size
t = tf.keras.preprocessing.text.Tokenizer(num_words=desired_vocab_size, oov_token=32) # num_words -> Vocablury size

In [18]:
#Fit tokenizer with actual training data
t.fit_on_texts(X_train.tolist())

In [19]:
#Vocabulary
t.word_index

{32: 1,
 'the': 2,
 'to': 3,
 'a': 4,
 'and': 5,
 'i': 6,
 'of': 7,
 'is': 8,
 'you': 9,
 'that': 10,
 'in': 11,
 'it': 12,
 'for': 13,
 'this': 14,
 'on': 15,
 'be': 16,
 'but': 17,
 'with': 18,
 'have': 19,
 'not': 20,
 'are': 21,
 'they': 22,
 'was': 23,
 'just': 24,
 'so': 25,
 'my': 26,
 'if': 27,
 'like': 28,
 'he': 29,
 'what': 30,
 'as': 31,
 'all': 32,
 "it's": 33,
 'or': 34,
 'at': 35,
 'would': 36,
 'do': 37,
 'people': 38,
 'your': 39,
 'get': 40,
 'me': 41,
 'we': 42,
 'can': 43,
 "don't": 44,
 'about': 45,
 'no': 46,
 'from': 47,
 'one': 48,
 'because': 49,
 'an': 50,
 'out': 51,
 'up': 52,
 "i'm": 53,
 'how': 54,
 'will': 55,
 'there': 56,
 'more': 57,
 'when': 58,
 'who': 59,
 'them': 60,
 'has': 61,
 'their': 62,
 'think': 63,
 'know': 64,
 'his': 65,
 'by': 66,
 'only': 67,
 'some': 68,
 'time': 69,
 'good': 70,
 'really': 71,
 'why': 72,
 'game': 73,
 'than': 74,
 'well': 75,
 'even': 76,
 'now': 77,
 'then': 78,
 'should': 79,
 'had': 80,
 'him': 81,
 'been': 82,
 '

# Prepare Training and Test Data

Get the word index for each of the word in the review

In [20]:
X_train[0:1]

9839    "black shield", xpecial confirmed racist and i...
dtype: object

In [21]:
X_train = t.texts_to_sequences(X_train.tolist())

In [22]:
print(X_train[0:1])

[[295, 1630, 9908, 1232, 459, 5, 4922, 45, 65, 1, 171, 3646, 9908, 414, 729]]


In [23]:
X_test = t.texts_to_sequences(X_test)

# Pad Sequences - Important

In [24]:
#Define maximum number of words to consider in each review
max_review_length = 300

In [25]:
#Pad training and test reviews
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                        maxlen=max_review_length,
                                                        padding='pre', 
                                                        truncating='post')

X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, 
                                                       maxlen=max_review_length, 
                                                       padding='pre',
                                                       truncating='post')

In [26]:
X_train.shape,X_test.shape

((12000, 300), (3000, 300))

In [27]:
X_train[100]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

# Build the Graph

In [28]:
tf.random.set_seed(42)

In [29]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

Add Embedding layer
 - Embedding Layer Input = Batch_Size * Length of each review

In [30]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    input_length=max_review_length) #Number of words in each review
          )

In [31]:
model.input

<KerasTensor: shape=(None, 300) dtype=float32 (created by layer 'embedding_input')>

In [32]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

Embedding Layer Output - [Batch_Size , Review Length , Embedding_Size]

Flatten the Output

In [33]:
model.add(tf.keras.layers.Flatten())

In [34]:
model.output

<KerasTensor: shape=(None, 15000) dtype=float32 (created by layer 'flatten')>

Add Dense Layer

In [35]:
model.add(tf.keras.layers.BatchNormalization())

In [36]:
model.output

<KerasTensor: shape=(None, 15000) dtype=float32 (created by layer 'batch_normalization')>

In [37]:
model.add(tf.keras.layers.Dense(128, activation='relu'))

In [38]:
model.output

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense')>

In [39]:
model.add(tf.keras.layers.Dropout(0.25))

In [40]:
model.output

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dropout')>

Use Dense layer for output layer

In [41]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [42]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 flatten (Flatten)           (None, 15000)             0         
                                                                 
 batch_normalization (BatchN  (None, 15000)            60000     
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 128)               1920128   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                        

# Execute the graph

In [44]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 12s 30ms/step - loss: 0.7161 - accuracy: 0.5284 - val_loss: 0.6902 - val_accuracy: 0.5470
Epoch 2/5
375/375 [==============================] - 11s 30ms/step - loss: 0.5014 - accuracy: 0.7513 - val_loss: 0.8710 - val_accuracy: 0.5560
Epoch 3/5
375/375 [==============================] - 13s 35ms/step - loss: 0.2114 - accuracy: 0.9173 - val_loss: 1.4055 - val_accuracy: 0.5473
Epoch 4/5
375/375 [==============================] - 12s 31ms/step - loss: 0.1112 - accuracy: 0.9620 - val_loss: 1.1563 - val_accuracy: 0.5470
Epoch 5/5
375/375 [==============================] - 11s 30ms/step - loss: 0.0795 - accuracy: 0.9754 - val_loss: 0.9851 - val_accuracy: 0.5580


Using Pre-Trained Embeddings

In [58]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [59]:
#Size of the model
wv.vectors.shape

(3000000, 300)

In [60]:
wv['india']

array([-2.34375000e-01, -7.17773438e-02,  1.05590820e-02,  3.26171875e-01,
       -6.29882812e-02, -1.78710938e-01,  3.17382812e-02, -3.96484375e-01,
       -1.69921875e-01, -3.54003906e-02, -1.81640625e-01, -3.28125000e-01,
        6.59179688e-02, -2.07031250e-01,  1.19140625e-01,  1.74804688e-01,
       -1.10839844e-01,  3.30078125e-01,  5.20019531e-02, -2.47802734e-02,
        1.48773193e-03, -1.60156250e-01,  2.70996094e-02, -1.80664062e-01,
       -4.14062500e-01,  1.95312500e-01, -3.49609375e-01,  1.03515625e-01,
       -8.54492188e-02, -1.48437500e-01, -8.25195312e-02, -2.90527344e-02,
       -3.02734375e-01,  1.98974609e-02, -3.26171875e-01,  1.70898438e-01,
       -4.55078125e-01, -4.39453125e-03,  4.27734375e-01, -2.13867188e-01,
       -6.86645508e-03,  1.23535156e-01,  4.96093750e-01,  3.41796875e-01,
        1.70898438e-01, -1.56250000e-01, -9.42382812e-02, -5.73730469e-02,
       -1.95312500e-01,  6.44531250e-02, -1.49414062e-01,  1.58203125e-01,
        2.53906250e-01,  

In [61]:
#Initialize embedding matrix for our dataset with 10000+1 rows (1 for padding word)
#and 50 columns (as embedding size is 50)
embedding_matrix = np.zeros((desired_vocab_size + 1, 50))

In [62]:
embedding_matrix.shape

(10001, 50)

In [63]:
embedding_matrix[200]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [64]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > (desired_vocab_size+1):
        break
    try:
        embedding_vector = wv[word] #Reading word's embedding from Glove model for a given word
        embedding_matrix[i] = embedding_vector
    except:
        pass

In [65]:
embedding_matrix[200]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Building a model using Pre-Trained embeddings

In [66]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [67]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    weights=[embedding_matrix],
                                    trainable=False,
                                    input_length=max_review_length) #Number of words in each review
          )

In [68]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

In [69]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [70]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [71]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 flatten (Flatten)           (None, 15000)             0         
                                                                 
 batch_normalization (BatchN  (None, 15000)            60000     
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 128)               1920128   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                        

In [72]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 9s 23ms/step - loss: 0.6932 - accuracy: 0.4982 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 2/5
375/375 [==============================] - 9s 23ms/step - loss: 0.6932 - accuracy: 0.5017 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 3/5
375/375 [==============================] - 9s 23ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 4/5
375/375 [==============================] - 10s 27ms/step - loss: 0.6932 - accuracy: 0.4992 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 5/5
375/375 [==============================] - 9s 23ms/step - loss: 0.6932 - accuracy: 0.5017 - val_loss: 0.6931 - val_accuracy: 0.5023


# With Simple RNN

In [73]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [74]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    weights=[embedding_matrix],
                                    trainable=False,
                                    input_length=max_review_length) #Number of words in each review
          )

In [75]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

Add Simple RNN layer

In [76]:
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.SimpleRNN(128, activation='relu')) #RNN State - size of memory

In [77]:
model.output

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'simple_rnn')>

In [78]:
model.add(tf.keras.layers.Dropout(0.25))

Output layer

In [79]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [80]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [81]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 batch_normalization (BatchN  (None, 300, 50)          200       
 ormalization)                                                   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               22912     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 523,291
Trainable params: 23,141
Non-trainable params: 500,150
_____________________________________________

In [82]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 33s 86ms/step - loss: 0.6932 - accuracy: 0.4982 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 2/5
375/375 [==============================] - 34s 90ms/step - loss: 0.6932 - accuracy: 0.5017 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 3/5
375/375 [==============================] - 33s 87ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 4/5
375/375 [==============================] - 39s 104ms/step - loss: 0.6932 - accuracy: 0.4992 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 5/5
375/375 [==============================] - 39s 105ms/step - loss: 0.6932 - accuracy: 0.5017 - val_loss: 0.6931 - val_accuracy: 0.5023


In [83]:
model.fit(X_train,y_train,
          epochs=10,
          initial_epoch=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 6/10
375/375 [==============================] - 33s 88ms/step - loss: 0.6932 - accuracy: 0.4943 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 7/10
375/375 [==============================] - 34s 90ms/step - loss: 0.6932 - accuracy: 0.5017 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 8/10
375/375 [==============================] - 33s 87ms/step - loss: 0.6932 - accuracy: 0.5002 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 9/10
375/375 [==============================] - 33s 87ms/step - loss: 0.6932 - accuracy: 0.4985 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 10/10
375/375 [==============================] - 33s 87ms/step - loss: 0.6932 - accuracy: 0.5017 - val_loss: 0.6931 - val_accuracy: 0.5023


In [84]:
from sklearn.metrics import accuracy_score, recall_score, classification_report, cohen_kappa_score
from sklearn import metrics 

In [85]:
####### Prediction ##########
predictions = model.predict(X_test)

94/94 [==============================] - 2s 21ms/step


In [86]:
predictions1 = [0 if x < 0.5 else 1 for x in predictions]

In [87]:
print('Baseline: Accuracy: ', round(accuracy_score(y_test, predictions1)*100, 2))
print('\n Classification Report:\n', classification_report(y_test,predictions1))

Baseline: Accuracy:  50.23

 Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      1493
           1       0.50      1.00      0.67      1507

    accuracy                           0.50      3000
   macro avg       0.25      0.50      0.33      3000
weighted avg       0.25      0.50      0.34      3000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Using LSTM

In [88]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [89]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    weights=[embedding_matrix],
                                    trainable=False,
                                    input_length=max_review_length) #Number of words in each review
          )

In [90]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

In [91]:
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LSTM(128)) #RNN State - size of memory

In [92]:
model.output

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>

In [93]:
model.add(tf.keras.layers.Dropout(0.25))

In [94]:
#Output
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [95]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [96]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 batch_normalization (BatchN  (None, 300, 50)          200       
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 128)               91648     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 592,027
Trainable params: 91,877
Non-trainable params: 500,150
_____________________________________________

In [97]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 143s 377ms/step - loss: 0.6934 - accuracy: 0.4994 - val_loss: 0.6932 - val_accuracy: 0.5023
Epoch 2/5
375/375 [==============================] - 140s 374ms/step - loss: 0.6932 - accuracy: 0.5066 - val_loss: 0.6932 - val_accuracy: 0.4977
Epoch 3/5
375/375 [==============================] - 140s 374ms/step - loss: 0.6934 - accuracy: 0.4967 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 4/5
375/375 [==============================] - 140s 374ms/step - loss: 0.6934 - accuracy: 0.4968 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 5/5
375/375 [==============================] - 140s 374ms/step - loss: 0.6933 - accuracy: 0.5010 - val_loss: 0.6931 - val_accuracy: 0.5023


In [99]:
####### Prediction ##########
predictions = model.predict(X_test)

94/94 [==============================] - 14s 149ms/step


In [100]:
predictions1 = [0 if x < 0.5 else 1 for x in predictions]

In [101]:
print('Baseline: Accuracy: ', round(accuracy_score(y_test, predictions1)*100, 2))
print('\n Classification Report:\n', classification_report(y_test,predictions1))

Baseline: Accuracy:  49.77

 Classification Report:
               precision    recall  f1-score   support

           0       0.50      1.00      0.66      1493
           1       0.00      0.00      0.00      1507

    accuracy                           0.50      3000
   macro avg       0.25      0.50      0.33      3000
weighted avg       0.25      0.50      0.33      3000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Using BiLSTM

In [115]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [116]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    weights=[embedding_matrix],
                                    trainable=False,
                                    input_length=max_review_length) #Number of words in each review
          )

In [117]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

In [118]:
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, return_sequences=True), merge_mode='sum'))

In [119]:
model.output

<KerasTensor: shape=(None, 300, 200) dtype=float32 (created by layer 'bidirectional')>

In [107]:
#LSTM2
#model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True), merge_mode='concat'))

In [108]:
#model.output

<KerasTensor: shape=(None, 300, 300) dtype=float32 (created by layer 'bidirectional_1')>

In [109]:
#LSTM3
#model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100), merge_mode='mul'))

In [110]:
#model.output

<KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'bidirectional_2')>

In [120]:
model.add(Dropout(0.5))
model.add(Dense(1,activation = 'sigmoid'))

In [121]:
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [122]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 batch_normalization (BatchN  (None, 300, 50)          200       
 ormalization)                                                   
                                                                 
 bidirectional (Bidirectiona  (None, 300, 200)         401600    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 300, 200)          0         
                                                                 
 dense (Dense)               (None, 300, 1)            201       
                                                                 
Total params: 902,051
Trainable params: 401,901
Non-trai

In [123]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 454s 1s/step - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6931 - val_accuracy: 0.4977
Epoch 3/5
375/375 [==============================] - 453s 1s/step - loss: 0.6933 - accuracy: 0.4945 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 4/5
375/375 [==============================] - 457s 1s/step - loss: 0.6933 - accuracy: 0.4989 - val_loss: 0.6931 - val_accuracy: 0.5023
Epoch 5/5
375/375 [==============================] - 452s 1s/step - loss: 0.6932 - accuracy: 0.4969 - val_loss: 0.6931 - val_accuracy: 0.5023


In [124]:
####### Prediction ##########
predictions = model.predict(X_test)

94/94 [==============================] - 32s 327ms/step


In [134]:
predictions1 = [0 if x.any() < 0.5 else 1 for x in predictions]

In [135]:
print('Baseline: Accuracy: ', round(accuracy_score(y_test, predictions1)*100, 2))
print('\n Classification Report:\n', classification_report(y_test,predictions1))

Baseline: Accuracy:  50.23

 Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      1493
           1       0.50      1.00      0.67      1507

    accuracy                           0.50      3000
   macro avg       0.25      0.50      0.33      3000
weighted avg       0.25      0.50      0.34      3000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
